[Back to Overview](overview.ipynb)

# CALLING CALLABLES

An exercise I usually walk us through, as a strong introduction to using `__call__`, is I'll take a function type object had have a class swallow it though `__init__` (birth method) but have the instance be a callable into that function.

In [1]:
class Compose:
    
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *p, **k):
        return self.func(*p, **k)
    
def parabola(x, c=0):
    return x ** 2 + c

parabola = Compose(parabola)  # what 'decorating' does
print(type(parabola))
print(list(map(parabola, range(-5, 6))))

<class '__main__.Compose'>
[25, 16, 9, 4, 1, 0, 1, 4, 9, 16, 25]


This is precisely the use case for decorator syntax...

In [2]:
@Compose
def trinomial(x, A=1, B=1, C=0):
    return A * x ** 2 + B * x + C

print(type(trinomial))
print(list(map(trinomial, range(-5, 6))))

<class '__main__.Compose'>
[20, 12, 6, 2, 0, 0, 2, 6, 12, 20, 30]


However, before we talk about decorators, we need to focus on:

```python
    def __call__(self, *p, **k):
        return self.func(*p, **k)
```

which is making use of the syntax we need to focus on.  The star, double-star stuff.

When part of a `def`, we define parameters.  Once our callable is defined, we pass it arguments.  Parameters on the receiving end need to match up with arguments and bring them in to the function body.  The passed in arguments may acquire new names at this threshold, to serve under those names in the function body.

Parameters do not join the global namespace.  They welcome arguments at the door, one might say, and usher them in to the scope of work.

Below, we confirm no Python name `n` has been defined, neither before, nor after we have used `n` in the definition of a function, as a parameter.

In [3]:
print("n" in globals())

False


In [4]:
carton = ['egg{}'.format(x) for x in range(12)]

def get_from_carton(n=0):
    return carton[n]

print(get_from_carton(6))

print("n" in globals())

egg6
False


In the above example, you're free to replace the 6 with any integer from 0 through 11, a way of addressing and getting back a reference to any egg in the global `carton`.  That choice of integer than gains the name `n` upon crossing the threshold into the `get_from_carton` function.

However suppose we wish to retrieve an arbitrary number of eggs in arbitrary order.  How might we do that?

In [5]:
def get_from_carton(*sel):
    tray = []
    for idx in sel:
        tray.append(carton[idx])
    return tray

print(get_from_carton(6, 3, 1, 11, 0))

['egg6', 'egg3', 'egg1', 'egg11', 'egg0']


What the star in front of `sel` accomplished is the `gathering up` of five separately passed positional arguments, into a single tuple, named `sel`.  Even had only a single number been passed, `*sel` would have turned it into a tuple, of one element.  No matter, things work out.

A for loop needs an iterable in all cases that an iterable, more specifically a tuple, is exactly what `*sel` provides.

The double star serves as a parallel unary operator in the case of named arguments.  Any named arguments that don't match with either positionals or named parameters, get added to a catch-all dictionary.  We may study this phenomenon in vitro:

In [6]:
def fancy(a, b, c=1, d=2, *, e, **k):
    return "a={}; b={}; c={}; d={}, e={}, k={}".format(a,b,c,d,e,k)
    
result = fancy(b=2, s=9, a=10, e=11, r=5)
print(result)

a=10; b=2; c=1; d=2, e=11, k={'s': 9, 'r': 5}


`a` and `b` get passed by name, in no particular order, yet match up with their positional counterparts just fine.  The star all by itself is a subtlety, and means the sebsequent parameters must be addressed by name, yet need not, as parameters, have default values.  We must pass an `e` by name, or get a `TypeError`.

However, the main feature to which I draw your attention is the `**k`.  This is the "gatherer of all wayward named arguments", meaning the `s=9` and `r=5` in the function call, have somewhere to go:  into a dictionary named `k`.  

Often, a Pythonista will use the specific names `args` and `kwargs` in connection with star and double star respectively.

```python
    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)
```

The above version of `__call__` has been slightly modified to match the aforementioned convention.

So now that we have explored the star (make a tuple) and double star (make a dict), when used at in the parameter definition, what does it mean to use them with arguments.

Remember, arguments go into the callable's "mouth" when we call it, only to match up with parameters earlier defined.

Quite rationally, star and double star perform complementary operations on the argument side.  Instead of gathering up, they break apart, or "explode" types of objects.  

A single star explodes sequences, such as tuples and lists, ranges and strings, into separate (individual) positional arguments.

A double star does the same for a dict type, exploding it into individual named arguments.

In [7]:
def get_letters(*s):  # all the letters, gathered into a tuple
    return s          # ... and simply returned

print(get_letters(*"a single string but with a star in front"))

('a', ' ', 's', 'i', 'n', 'g', 'l', 'e', ' ', 's', 't', 'r', 'i', 'n', 'g', ' ', 'b', 'u', 't', ' ', 'w', 'i', 't', 'h', ' ', 'a', ' ', 's', 't', 'a', 'r', ' ', 'i', 'n', ' ', 'f', 'r', 'o', 'n', 't')


In [8]:
def get_names(*, a, b):
    return a + b

get_names(**{"a":10, "b":11})

21

In [9]:
get_names(b=11, a=10)

21

And that, my friends, rounds out the mystery.  The star and double star each appear in two roles: in front of parameters; in front of arguments.

In front of parameters, they gather up, vacuum up, hoover.  In front of arguments, they break apart, fragment, explode.

A dict is a payload of named arguments, but needs to be unpacked.  Double star is the unpacker.  These unpacked named arguments may then be met by a double starred parameter on the receiving end.  There, the named arguments roll back up into a dictionary, and so on.

The economy of this interface is time tested and keeps Python flexible and sinewy.